In [1]:
addprocs(CPU_CORES)

@everywhere begin
  using StochasticDiffEq, DiffEqProblemLibrary, ParallelDataTransfer
  srand(99 + myid())
  prob = oval2ModelExample(largeFluctuations=true,useBigs=false)
  prob.tspan = (0.0,1.0)
  println("Solve once to compile.")
  sol = solve(prob,EM(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,RKMil(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,SRIW1(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,SRI(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt1()),dt=1/2^(18),adaptive=false,save_timeseries=false)
  Int(sol.u[1]!=NaN)
  println("Compilation complete.")
  js = 16:21
  dts = 1./2.^(js)
  fails = Array{Int}(length(dts),3)
  times = Array{Float64}(length(dts),3)
  numRuns = 10000
end
println("Setup Complete")

  likely near In[1]:1
#algType<:DiffEqBase.AbstractSDEAlgorithm)WARNING: Method definition #init in module StochasticDiffEq(Array at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10{Any overwritten,  in module StochasticDiffEq1 at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10}, .
DiffEqBase.#init, DiffEqBase.WARNING: Method definition AbstractSDEProblemsolve{(#uType<:DiffEqBaseAny., AbstractSDEProblem#tType<:{Any, #uType<:#isinplace<:AnyAny, , #tType<:#NoiseClass<:AnyAny, , #F<:Any#isinplace<:, Any#F2<:Any, , #NoiseClass<:#F3<:AnyAny, }, #F<:#algType<:AnyDiffEqBase, .AbstractSDEAlgorithm#F2<:, AnyAny, , #F3<:AnyAny)} in module StochasticDiffEq,  overwritten in module StochasticDiffEq#algType<:.
DiffEqBase.AbstractSDEAlgorithm, WARNING: Method definition Any#init)( in module StochasticDiffEqArray at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10{ overwrittenAny in module StochasticDiffEq, 1 at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl

	From worker 11:	Solve once to compile.
	From worker 16:	Solve once to compile.
	From worker 3:	Solve once to compile.
	From worker 10:	Solve once to compile.
	From worker 9:	Solve once to compile.
	From worker 8:	Solve once to compile.
	From worker 17:	Solve once to compile.
	From worker 12:	Solve once to compile.
	From worker 5:	Solve once to compile.
	From worker 14:	Solve once to compile.
	From worker 2:	Solve once to compile.
	From worker 4:	Solve once to compile.
	From worker 7:	Solve once to compile.
	From worker 6:	Solve once to compile.
	From worker 15:	Solve once to compile.
	From worker 13:	Solve once to compile.
	From worker 16:	Compilation complete.
	From worker 11:	Compilation complete.
	From worker 3:	Compilation complete.
	From worker 9:	Compilation complete.
	From worker 10:	Compilation complete.
	From worker 4:	Compilation complete.
	From worker 14:	Compilation complete.
	From worker 5:	Compilation complete.
	From worker 7:	Compilation complete.
	From worker 8:	Compil

Array, {1}Symbol, , 1Float64}, , Float64Array, {Float64SymEngine). in module ParameterizedFunctionsBasic,  at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/maketype.jl:25 overwritten1} at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/maketype.jl:25, .
Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Expr, Array{Symbol, 1}, Float64, Float64, Float64) in module ParameterizedFunctions at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/maketype.jl:25 overwritten at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/maketype.jl:25.
, Any, Any) in module ParameterizedFunctions at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/ode_def_opts.jl:237 overwritten at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/ode_def_opts.jl:237.
{An

In [8]:
## Timing Runs

@everywhere function runAdaptiveSRIW1(i)
  sol = solve(prob,SRIW1(),abstol=2.0^(-20),reltol=2.0^(-14),maxIters=Int(1e11),qmax=1.125,save_timeseries=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIW1,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 42.287044095


In [6]:
## Timing Runs

@everywhere function runAdaptiveSRI(i)
  sol = solve(prob,SRI(),abstol=2.0^(-20),reltol=2.0^(-15),maxIters=Int(1e11),qmax=1.125,save_timeseries=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRI,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 186.292485925


In [12]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt1(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt1()),abstol=2.0^(-17),reltol=2.0^(-14),maxIters=Int(1e11),qmax=1.125,save_timeseries=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt1,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 97.670864661


In [5]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt2(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt2()),abstol=2.0^(-18),reltol=2.0^(-15),maxIters=Int(1e11),qmax=1.125,save_timeseries=false)
    Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt2,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 189.862145778


In [6]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt3(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt3()),abstol=2.0^(-16),reltol=2.0^(-14),maxIters=Int(1e10),qmax=1.125,save_timeseries=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt3,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 70.84360428


In [7]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt4(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt4()),abstol=2.0^(-14),reltol=2.0^(-10),maxIters=Int(1e10),qmax=1.125,save_timeseries=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt4,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 2. Elapsed time was 12.551133089


In [ ]:
@everywhere function runEM(i,j)
  sol =solve(prob,EM(),dt=dts[j],maxIters=Int(1e11),save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in eachindex(js)
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t1 = @elapsed numFails = sum(pmap((i)->runEM(i,j),1:numRuns))
  println("The number of Euler-Maruyama Fails is $numFails. Elapsed time was $t1")
  fails[j,1] = numFails
  times[j,1] = t1
end

j = 1
The number of Euler-Maruyama Fails is 152. Elapsed time was 48.540552449
j = 2
The number of Euler-Maruyama Fails is 35. Elapsed time was 96.719737224
j = 3
The number of Euler-Maruyama Fails is 4. Elapsed time was 195.506232676
j = 4
The number of Euler-Maruyama Fails is 0. Elapsed time was 389.441266894
j = 5
The number of Euler-Maruyama Fails is 0. Elapsed time was 775.631307613
j = 6


In [ ]:
@everywhere function runSRI(i,j)
  sol =solve(prob,SRIW1(),dt=dts[j],maxIters=Int(1e11),adaptive=false,save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in 1:4
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t2 = @elapsed numFails = sum(pmap((i)->runSRI(i,j),1:numRuns))
  println("The number of Rossler-SRI Fails is $numFails. Elapsed time was $t2")
  fails[j,2] = numFails
  times[j,2] = t2
end

In [ ]:
@everywhere function runMil(i,j)
  sol =solve(prob,RKMil(),dt=dts[j],maxIters=Int(1e11),save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in eachindex(js)
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t3 = @elapsed numFails = sum(pmap((i)->runMil(i,j),1:numRuns))
  println("The number of RK-Milstein Fails is $numFails. Elapsed time was $t3")
  fails[j,3] = numFails
  times[j,3] = t3
end

In [ ]:
using Plots
lw = 3
p2 = plot(dts,times,xscale=:log2,yscale=:log2,guidefont=font(16),tickfont=font(14),yguide="Elapsed Time (s)",xguide=L"Chosen $\Delta t$",top_margin=50px,linewidth=lw,lab=["Euler-Maruyama" "RK-Mil" "RosslerSRI"],legendfont=font(14))
plot!(dts,repmat([adaptiveTime],11),linewidth=lw,line=:dash,lab="ESRK+RSwM3",left_margin=75px)
scatter!([2.0^(-20);2.0^(-20);2.0^(-18)],[times[5,1];times[5,2];times[3,3]],markersize=20,c=:red,lab="")
plot(p2,size=(800,800))